# Preprocessing Videos

In [4]:
# Get video list, and convert to picture
import os

folder_path = '../Dataset/video'
file_names = [files for files in os.listdir(folder_path) if files.endswith(".mp4")]

print("Video List:")
print(file_names)

path_list = []
for name in file_names:
    path_list.append(os.path.join(folder_path, name))

print("Video Paths:")
print(path_list)


Video List:
['Communal1.mp4', 'Communal2.mp4', 'VA1.mp4', 'VA2.mp4', 'VA3.mp4', 'VA4.mp4']
Video Paths:
['../Dataset/video\\Communal1.mp4', '../Dataset/video\\Communal2.mp4', '../Dataset/video\\VA1.mp4', '../Dataset/video\\VA2.mp4', '../Dataset/video\\VA3.mp4', '../Dataset/video\\VA4.mp4']


# Extract Frames

In [11]:
# import cv2
# import os

# for video in path_list: 
#     video_name = video
#     print("Processing video: " + file_names[path_list.index(video)])

#     cap = cv2.VideoCapture(video_name)

#     time_skips = float(1000) #skip every 1 seconds. modify if need

#     count = 0
#     success,image = cap.read()
#     while success:
#         # save image
#         img_name =  file_names[path_list.index(video)][:-4] + "/frame%d.jpg" % count
#         cv2.imwrite("../Dataset/images/Original/" + img_name, image)

#         cap.set(cv2.CAP_PROP_POS_MSEC, (count*time_skips))
#         print('New frame captured: ' + str(count))

#         # move the time
#         success,image = cap.read()
#         count += 1

#     # release after reading    
#     cap.release()

# print("Finish Processing all videos.")

# Preprocessing label with image-frames

In [7]:
# Processing data and label in every frame
import jsonlines # read annotation from .jsonl file
import os
import fnmatch

# load images
root = "../Dataset/label"
label_files = [files for files in os.listdir(root) if files.endswith(".jsonl")]

# print("Video List:")
# print(file_names)

customerNum = []
for file in label_files: 
    customer = []
    # print(file)
    file = os.path.join(root, file)
    if os.path.exists(file):
        # print('File exists')
        with jsonlines.open(file) as reader:
            # save annotation and file name into list
            for line in reader:
                num = line['customernum']
                customer.append(num)
    else:
        raise ValueError('Invalid label file path [%s]'%file)
    customerNum.append(customer)

print(customerNum)

# Get number of frames in every video
folder_dict = {}
for path, dirs, files in os.walk('../Dataset/images/Original'):
    folder_name = os.path.basename(path)
    file_count = len(fnmatch.filter(files, '*.jpg'))
    folder_dict[folder_name] = file_count

print(folder_dict) # number of frames



[[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3], [7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10], [2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 4, 4, 4, 4, 4, 4], [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6

In [4]:
from collections import deque

# Create a dictionary to keep track of the start and end frames for each person
person_frames = deque()
person_counter = 0

for customer in customerNum:
    i = 0
    for num_people in customer:
        
        # If there are more people than we're currently tracking, add new people
        while num_people > len(person_frames):
            person_counter += 1
            person_frames.append([person_counter, i, None])

        # If there are fewer people than we're currently tracking, update the end frames for the people who left
        while num_people < len(person_frames):
            # Update the end frame for the person who left
            person_frames.popleft()[2] = i - 1

        i += 1

    # Update the end frames for the remaining people
    for person_frame in person_frames:
        if person_frame[2] is None:
            person_frame[2] = i - 1

    # Calculate the processing time for each person
    processing_times = {person: end - start for person, start, end in person_frames}
    print(processing_times)

    # Clear the person frames for the next customer
    person_frames.clear()

{3: 53, 4: 53, 5: 53}
{8: 71, 9: 71, 10: 71, 11: 71, 12: 71, 13: 56, 14: 55, 15: 42, 16: 24, 17: 7}
{19: 24}
{25: 87, 26: 59, 27: 47, 28: 26}
{32: 89, 33: 89, 34: 79, 35: 50, 36: 24}
{42: 88, 43: 88, 44: 66, 45: 39, 46: 19, 47: 15, 48: 10, 49: 3}


# Serving Time Calculation

In [14]:
import numpy as np
from sklearn.linear_model import LinearRegression

# def serving_time(customerNum):
#     serving_times = []

#     for customer_list in customerNum:
#         serving_time = []
#         current_count = customer_list[0]
#         count_duration = 0

#         for count in customer_list:
#             if count == current_count:
#                 count_duration += 1
#             else:
#                 serving_time.append((current_count, count_duration))
#                 current_count = count
#                 count_duration = 1

#         # Append the last count
#         serving_time.append((current_count, count_duration))
#         serving_times.append(serving_time)

#     return serving_times

# time = serving_time(customerNum)



# # data = [(5, 10), (6, 19), (5, 10), (6, 26), (7, 5), (6, 15), (5, 5)]

# # data = [(5, 10), (6, 19), (5, 10), (6, 26), (7, 5), (6, 15), (5, 5)]

# def process_time(time_list):
#     converted_data = []
#     prev_key = None
#     prev_value = 0
#     for key, value in time_list:
#         if key == prev_key:
#             prev_value += value
#         else:
#             if prev_key is not None:
#                 converted_data.append((prev_key, prev_value))
#             prev_key = key
#             prev_value = value
#     converted_data.append((prev_key, prev_value))
#     print(converted_data)
#     # [(5, 10), (6, 19), (5, 10), (6, 26), (7, 5), (6, 15), (5, 5)]

# time_data = []
# # get serving time for 1 people
# for file in file_names:
#     time_list = time[file_names.index(file)]
#     print(file[:-4])
#     print(time_list)
#     # calculate the serving time for 1 people
#     processing_times = process_time(time_list)
#     time_data.append(processing_times)

#     # print(processing_times)

# print(time_data)


In [15]:

data = [(5, 10), (6, 19), (5, 10), (6, 26), (7, 5), (6, 15), (5, 5)]


# data[(1,10+19),(2,19+10+26+5+15),(3,10+26+5+15+5)]

# process_time(data)
# data = [(5, 10), (6, 19), (5, 10), (6, 26), (7, 5), (6, 15), (5, 5)]

# Initialize the cumulative sum
cumulative_sum = 0

# Create a new list to store the converted data
converted_data = []

for i, (_, value) in enumerate(data, start=1):
    # Add the current value to the cumulative sum
    cumulative_sum += value

    # Add the current index and the cumulative sum to the converted data
    converted_data.append((i, cumulative_sum))

print(converted_data)


[(1, 10), (2, 29), (3, 39), (4, 65), (5, 70), (6, 85), (7, 90)]


# HOG + SVM

### detecting people

In [5]:
import cv2

# Initialize HOG person detector
hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

pred_customer_num = []
for video in path_list: 
    print("Processing video: " + file_names[path_list.index(video)])

    # Open video
    cap = cv2.VideoCapture(video)

    time_skips = float(1000) #skip every 1 seconds. modify if need

    count = 0
    success,image = cap.read()
    pred_num_people = []
    while success:
            
        # Detect people in the image
        boxes, weights = hog.detectMultiScale(image, winStride=(4, 4), padding=(8, 8), scale=1.05)

        # Draw rectangle around each person
        for (x, y, w, h) in boxes:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
        
        # save image
        img_name =  file_names[path_list.index(video)][:-4] + "/frame%d.jpg" % count
        cv2.imshow("/frame%d" % count, image) 
        
        # Count the number of people detected
        num_people = len(boxes)
        print('frame:' + str(count), 'Detect people:', num_people)

        pred_num_people.append(num_people)
        
        
        cap.set(cv2.CAP_PROP_POS_MSEC, (count*time_skips))

        # move the time
        success,image = cap.read()
        count += 1

    pred_customer_num.append(pred_num_people)
    # release after reading    
    cap.release()

print("Finish Processing all videos.")


Processing video: Communal1.mp4
frame:0 Detect people: 10
frame:1 Detect people: 10
frame:2 Detect people: 8
frame:3 Detect people: 8
frame:4 Detect people: 11
frame:5 Detect people: 10
frame:6 Detect people: 13
frame:7 Detect people: 9
frame:8 Detect people: 14
frame:9 Detect people: 8
frame:10 Detect people: 8
frame:11 Detect people: 6
frame:12 Detect people: 13
frame:13 Detect people: 15
frame:14 Detect people: 13
frame:15 Detect people: 17
frame:16 Detect people: 15
frame:17 Detect people: 22
frame:18 Detect people: 20
frame:19 Detect people: 19
frame:20 Detect people: 12
frame:21 Detect people: 15
frame:22 Detect people: 16
frame:23 Detect people: 10
frame:24 Detect people: 12
frame:25 Detect people: 15
frame:26 Detect people: 16
frame:27 Detect people: 15
frame:28 Detect people: 12
frame:29 Detect people: 16
frame:30 Detect people: 16
frame:31 Detect people: 19
frame:32 Detect people: 20
frame:33 Detect people: 20
frame:34 Detect people: 17
frame:35 Detect people: 17
frame:36 Det

: 

### Match Rate for all algorithm

In [10]:
# Match Rate
def match_rate(original, pred):
    match_rate = []
    for i in range(len(original)):
        match = 0
        for j in range(len(original[i])):
            if original[i][j] == pred[i][j]:
                match += 1
        rate = match / len(original[i])
        match_rate.append(rate)
    return match_rate

### Accuracy for HOG + SVM

In [11]:
print("Match Rate for 6 videos for HOG:")
print(match_rate(customerNum, pred_customer_num))

Match Rate for 6 videos for HOG:


NameError: name 'pred_customer_num' is not defined

# YOLOv5
Using YOLOv5 model to detect people in picture

In [4]:
# convert into img path list and then sort by number in file name
import os

frame_path = '../Dataset/images/Original'
img_path_list = []

for root, dirs, files in os.walk(frame_path):
    temp_list = []
    for dir_name in dirs:
        # Create the full path to the directory
        dir_path = os.path.join(root, dir_name)
        # Get the names of all .jpg files in the directory
        for file in os.listdir(dir_path):
            if file.endswith(".jpg"):
                temp_list.append(os.path.join(dir_path, file))
        
        # Add the full paths to the .jpg files to temp_list
        # img_path_list.append(temp_list)
        # print(temp_list)
        temp_list.sort(key=lambda x: int(x[len(dir_path)+6:-4]))
        img_path_list.append(temp_list)
        temp_list = []
    # temp_list = 
    # print(temp_list)

# print(temp_list)    
# print(img_path_list[0])
# img_path_list = img_path_list[0]
# print(img_path_list)

['../Dataset/images/Original\\VA3\\frame0.jpg', '../Dataset/images/Original\\VA3\\frame1.jpg', '../Dataset/images/Original\\VA3\\frame2.jpg', '../Dataset/images/Original\\VA3\\frame3.jpg', '../Dataset/images/Original\\VA3\\frame4.jpg', '../Dataset/images/Original\\VA3\\frame5.jpg', '../Dataset/images/Original\\VA3\\frame6.jpg', '../Dataset/images/Original\\VA3\\frame7.jpg', '../Dataset/images/Original\\VA3\\frame8.jpg', '../Dataset/images/Original\\VA3\\frame9.jpg', '../Dataset/images/Original\\VA3\\frame10.jpg', '../Dataset/images/Original\\VA3\\frame11.jpg', '../Dataset/images/Original\\VA3\\frame12.jpg', '../Dataset/images/Original\\VA3\\frame13.jpg', '../Dataset/images/Original\\VA3\\frame14.jpg', '../Dataset/images/Original\\VA3\\frame15.jpg', '../Dataset/images/Original\\VA3\\frame16.jpg', '../Dataset/images/Original\\VA3\\frame17.jpg', '../Dataset/images/Original\\VA3\\frame18.jpg', '../Dataset/images/Original\\VA3\\frame19.jpg', '../Dataset/images/Original\\VA3\\frame20.jpg', '

In [5]:
# %cd content
# !git clone https://github.com/ultralytics/yolov5
# %cd yolov5
# %pip install -r requirements.txt

# from yolov5 import utils
# display = utils.notebook_init()

# !python detect.py --weights yolov5s.pt --img 256 --conf 0.25 --source ../Dataset/images/Original/Communal1

# reference: https://github.com/ultralytics/yolov5/issues/36

import torch

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

people_list = []
for img_path in img_path_list:
    temp_list = []
    for img in img_path:
        # # Image
        # im = '../Dataset/images/Original/Communal1/frame0.jpg'

        # Inference
        results = model(img)

        # print(results.pandas().xyxy[0])
        # results.pandas().xyxy[0]
        #      xmin    ymin    xmax   ymax  confidence  class    name
        # 0  749.50   43.50  1148.0  704.5    0.874023      0  person
        # 1  433.50  433.50   517.5  714.5    0.687988     27     tie
        # 2  114.75  195.75  1095.0  708.0    0.624512      0  person
        # 3  986.00  304.00  1028.0  420.0    0.286865     27     tie

        # Filter the DataFrame to only include rows where 'name' is 'person'
        people = results.pandas().xyxy[0][results.pandas().xyxy[0]['name'] == 'person']
        # confidence = people['confidence']
        # # Get the number of people
        num_people = len(people)
        # # Calculate the average confidence
        # average_confidence = confidence.mean()
        print('Number of people detected:', num_people)
        temp_list.append(num_people)

        # # print('Confidence:', confidence)
        # print('Average confidence:', average_confidence)
    print(temp_list)
    people_list.append(temp_list)

print(people_list)


Using cache found in C:\Users\zoezh/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-5-7 Python-3.9.7 torch-2.2.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3050 Ti Laptop GPU, 4096MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


Number of people detected: 8
Number of people detected: 8
Number of people detected: 7
Number of people detected: 8
Number of people detected: 9
Number of people detected: 9
Number of people detected: 8
Number of people detected: 9
Number of people detected: 10
Number of people detected: 9
Number of people detected: 9
Number of people detected: 8
Number of people detected: 8
Number of people detected: 9
Number of people detected: 8
Number of people detected: 8
Number of people detected: 9
Number of people detected: 9
Number of people detected: 7
Number of people detected: 7
Number of people detected: 7
Number of people detected: 7
Number of people detected: 6
Number of people detected: 6
Number of people detected: 7
Number of people detected: 6
Number of people detected: 5
Number of people detected: 6
Number of people detected: 5
Number of people detected: 5
Number of people detected: 5
Number of people detected: 5
Number of people detected: 6
Number of people detected: 5
Number of peo

### Detection Accuracy for YOLOv5 Model

In [10]:
print(match_rate(customerNum, people_list))

[0.018518518518518517, 0.19444444444444445, 0.16, 0.0, 0.4, 0.10112359550561797]


### Validation

In [ ]:
!python val.py --data /path/to/validation/data.yaml --weights /path/to/weights/best.pt --img 640 --conf 0.25 --iou 0.45

# Faster R-CNN

### Model Algorithm

In [5]:

import torchvision
import cv2
import numpy as np
import torch
import torchvision.transforms as T

# Load the pre-trained Faster R-CNN model

########################### cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model = model.to(device)  # Move the model to GPU

model.eval()

people_list_RCNN = []
# load video path for capture
for video in path_list: 
    # Load the video
    # cap = cv2.VideoCapture('../Dataset/video/VA1.mp4')
    cap = cv2.VideoCapture(video)

    # Initialize frame counter
    frame_counter = 0

    temp_list =[]
    print("Processing video:", video)
    while(cap.isOpened()):
        # Set the position in the video to the current second
        # cap.set(cv2.CAP_PROP_POS_MSEC, seconds * 1000)
        # Read the next frame
        ret, frame = cap.read()
        if ret == True:
            frame_model = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_model = frame_model / 255.0
            frame_model = np.transpose(frame_model, (2, 0, 1))
            frame_model = torch.from_numpy(frame_model).float().to(device)   ########################### cuda .to(device)

            # Apply the Faster R-CNN model to the frame
            output = model([frame_model])
            
            ########################### cuda
            # output = [{k: v.to('cpu') for k, v in t.items()} for t in output]

            # Apply non-maximum suppression
            nms_indices = torchvision.ops.nms(output[0]['boxes'], output[0]['scores'], 0.3)
            output[0]['boxes'] = output[0]['boxes'][nms_indices]
            output[0]['labels'] = output[0]['labels'][nms_indices]

            # Filter out the detections with low confidence scores
            high_conf_indices = [i for i, score in enumerate(output[0]['scores']) if score > 0.7]
            output[0]['boxes'] = output[0]['boxes'][high_conf_indices]
            output[0]['labels'] = output[0]['labels'][high_conf_indices]

            # Count the number of people detected in the frame
            num_people = sum(1 for box, label in zip(output[0]['boxes'], output[0]['labels']) if label == 1)

            # Draw bounding boxes around the detected people
            for box, label in zip(output[0]['boxes'], output[0]['labels']):
                if label == 1:
                    cv2.rectangle(frame, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0, 255, 0), 2)
            
            # Save the frame
            # cv2.imwrite(f'../Dataset/images/RCNN/frame{frame_counter}.jpg', frame)
            

            cap.set(cv2.CAP_PROP_POS_MSEC, frame_counter * 1000)
            # Increment frame counter
            frame_counter += 1

            # Print the number of people detected in the frame along with the frame number
            print(f'Frame {frame_counter}: Number of people: {num_people}')
            temp_list.append(num_people)
        else:
            break
        
    # Release the video capture
    cap.release()
    print(video, temp_list)
    people_list_RCNN.append(temp_list)

# Close all OpenCV windows
cv2.destroyAllWindows()

Processing video: ../Dataset/video\Communal1.mp4
Frame 1: Number of people: 7
Frame 2: Number of people: 7
Frame 3: Number of people: 7
Frame 4: Number of people: 8
Frame 5: Number of people: 8
Frame 6: Number of people: 7
Frame 7: Number of people: 8
Frame 8: Number of people: 8
Frame 9: Number of people: 7
Frame 10: Number of people: 8
Frame 11: Number of people: 8
Frame 12: Number of people: 7
Frame 13: Number of people: 8
Frame 14: Number of people: 7
Frame 15: Number of people: 8
Frame 16: Number of people: 8
Frame 17: Number of people: 9
Frame 18: Number of people: 8
Frame 19: Number of people: 7
Frame 20: Number of people: 7
Frame 21: Number of people: 7
Frame 22: Number of people: 6
Frame 23: Number of people: 6
Frame 24: Number of people: 6
Frame 25: Number of people: 6
Frame 26: Number of people: 5
Frame 27: Number of people: 5
Frame 28: Number of people: 6
Frame 29: Number of people: 5
Frame 30: Number of people: 5
Frame 31: Number of people: 4
Frame 32: Number of people: 6


### Acuuracy for faster RCNN algorithm

In [8]:
print(people_list_RCNN)
print("Match Rate for 6 videos for faster RCNN:")
print(len(people_list_RCNN[0]), len(customerNum[0]))
print(len(people_list_RCNN[1]), len(customerNum[1]))
print(len(people_list_RCNN[2]), len(customerNum[2]))
print(len(people_list_RCNN[3]), len(customerNum[3]))
print(len(people_list_RCNN[4]), len(customerNum[4]))
print(len(people_list_RCNN[5]), len(customerNum[5]))


# print(match_rate(customerNum, people_list_RCNN))

[[7, 7, 7, 8, 8, 7, 8, 8, 7, 8, 8, 7, 8, 7, 8, 8, 9, 8, 7, 7, 7, 6, 6, 6, 6, 5, 5, 6, 5, 5, 4, 6, 5, 4, 3, 4, 5, 6, 7, 6, 6, 6, 5, 6, 5, 6, 6, 7, 7, 6, 6, 9, 7, 7, 6, 4, 3], [9, 9, 9, 8, 8, 8, 8, 10, 9, 9, 8, 8, 9, 10, 13, 6, 13, 9, 11, 11, 12, 10, 10, 10, 11, 9, 8, 11, 13, 12, 12, 12, 7, 10, 8, 7, 7, 6, 5, 5, 7, 7, 7, 8, 6, 8, 11, 9, 7, 8, 9, 12, 9, 8, 12, 8, 8, 8, 8, 8, 7, 10, 6, 8, 7, 4, 3, 10, 8, 9, 9, 10, 7, 10, 10], [3, 3, 5, 3, 4, 4, 4, 4, 1, 2, 3, 3, 2, 1, 2, 1, 2, 3, 2, 1, 2, 3, 4, 4, 3], [10, 10, 8, 8, 10, 7, 10, 10, 9, 9, 8, 8, 7, 7, 7, 8, 8, 7, 8, 7, 8, 9, 7, 7, 7, 7, 11, 9, 9, 7, 8, 6, 8, 8, 8, 11, 9, 9, 8, 7, 7, 10, 8, 8, 7, 6, 7, 7, 6, 6, 7, 7, 7, 8, 9, 8, 9, 8, 8, 7, 8, 7, 7, 7, 7, 8, 9, 9, 9, 8, 7, 8, 11, 9, 10, 8, 8, 9, 8, 10, 9, 7, 8, 9, 10, 8, 9, 10, 8, 6, 7, 7, 7, 8, 7, 8, 6, 8, 8, 9, 9, 10, 10, 8, 10, 8, 9, 9, 10, 9, 9, 8, 8, 9, 6, 9, 6, 10, 9, 10, 10, 6, 9, 6, 11], [5, 5, 5, 5, 5, 5, 5, 5, 4, 5, 7, 5, 6, 7, 6, 6, 6, 6, 6, 6, 7, 6, 5, 7, 6, 8, 8, 8, 6, 8, 6, 8, 10

# Estimate average waiting time per window per canteen

In [ ]:
# estimated waiting time at 5-second intervals

